### This notebook shows examples of how to use the cyclops.processor API on GEMINI.

## Get all patient encounters including ER data, with diagnoses from St. Michael's hospital from March 1 2020 to March 15 2020, with vitals and labs and interventions.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

from cyclops.processor import featurize
from cyclops.processors.aggregate import Aggregator
from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    AGE,
    DIAGNOSIS_CODE,
    ENCOUNTER_ID,
    HOSPITAL_ID,
    SEX,
)
from cyclops.processors.constants import SMH
from cyclops.feature_handler import FeatureHandler
from cyclops.processors.impute import Imputer
from cyclops.processors.utils import gather_columns
from cyclops.query import gemini

# patients = gemini.patients(
#     hospitals=[SMH], from_date="2020-03-01", to_date="2020-03-15", include_er_data=True
# )
# patients_diagnoses = gemini.diagnoses(patients=patients)
# patients_vitals = gemini.events(patients=patients, category="vitals")
# patients_labs = gemini.events(patients=patients, category="lab")
# patients_interventions = gemini.events(patients=patients, category="intervention")

# diagnoses_data = patients_diagnoses.run()
# vitals_data = patients_vitals.run()
# labs_data = patients_labs.run()
# interventions_data = patients_interventions.run()

# print(f"{len(diagnoses_data)} diagnoses rows extracted!")
# print(f"{len(vitals_data)} vitals rows extracted!")
# print(f"{len(labs_data)} labs rows extracted!")
# print(f"{len(interventions_data)} interventions rows extracted!")

# patients_diagnoses.save(".", "diagnoses")
# patients_vitals.save(".", "vitals")
# patients_labs.save(".", "lab")
# patients_interventions.save(".", "intervention")

2022-04-29 10:57:12,627 INFO cyclops.utils.profile - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-29 10:57:12,664 INFO cyclops.processors.impute - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-29 10:57:12,943 INFO cyclops.feature_handler - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-29 10:57:12,962 INFO cyclops.processors.utils - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-29 10:57:12,991 INFO cyclops.processors.aggregate - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-29 10:57:13,030 INFO cyclops.processors.diagnoses - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-29 10:57:13,050 INFO cyclops.processors.events - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-29 10:57:13,083 INFO cyclops.processors.outcomes - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-29 10:57:13,255 INFO cyclops.config  - Log file

## Process queried data into features. (Static + Temporal).

In [2]:
diagnoses_data = pd.read_parquet("diagnoses.gzip")
vitals_data = pd.read_parquet("vitals.gzip")
labs_data = pd.read_parquet("lab.gzip")
interventions_data = pd.read_parquet("intervention.gzip")

static_diagnoses_data = gather_columns(
    diagnoses_data,
    [ENCOUNTER_ID, AGE, SEX, DIAGNOSIS_CODE, HOSPITAL_ID, ADMIT_TIMESTAMP],
)
feature_handler = featurize(
    static_data=[static_diagnoses_data],
    temporal_data=[labs_data, vitals_data],
    aggregator=Aggregator(bucket_size=1, window=48),
    static_imputer=Imputer(strategy="median"),
    temporal_imputer=Imputer(strategy="median"),
    reference_cols=[HOSPITAL_ID, ADMIT_TIMESTAMP],
)

2022-04-29 10:57:16,939 INFO cyclops.processors.utils - Processing raw diagnosis codes...
2022-04-29 10:57:16,946 INFO cyclops.processors.utils - # samples: 1068, # encounters: 150
2022-04-29 10:57:16,989 INFO cyclops.processors.utils - Grouping ICD codes to trajectories...
2022-04-29 10:57:16,991 INFO cyclops.processors.utils - # samples: 1068, # encounters: 150
2022-04-29 10:57:16,993 INFO cyclops.processors.diagnoses - # diagnosis features: 19, # encounters: 150
2022-04-29 10:57:17,078 INFO cyclops.utils.profile - Finished executing function group_diagnosis_codes_to_trajectories in 0.138797 s
2022-04-29 10:57:17,102 INFO cyclops.processors.utils - Gathering static features...
2022-04-29 10:57:17,104 INFO cyclops.processors.utils - # samples: 1068, # encounters: 150
2022-04-29 10:57:17,106 INFO cyclops.processors.utils - # columns: 5, # encounters: 150
2022-04-29 10:57:17,282 INFO cyclops.utils.profile - Finished executing function gather_static_features in 0.179345 s
2022-04-29 10:5

## Inspect feature handler, plot example temporal features and save them.

In [5]:
print(feature_handler.get_numerical_feature_names())
print(feature_handler.get_categorical_feature_names())

plot_features = [
    "oxygen saturation",
]

encounter_id = 11942185
feature_handler.plot_features(encounter_id, names=plot_features)
feature_handler.save(".", "test_features")

{'static': ['age'], 'temporal': ['sodium', 'urine sodium', 'urine specific gravity', 'bicarbonate', 'blood urea nitrogen', 'vitamin b12', 'white blood cell count', 'high sensitivity troponin', 'vitamin d', 'hba1c', 'lymphocyte', 'neutrophils', 'albumin', 'aptt', 'bilirubin', 'calcium', 'creatinine', 'calcium, ionized', 'esr', 'serum alcohol', 'ferritin', 'fibrinogen', 'glucose fasting', 'glucose point of care', 'glucose random', 'hematocrit', 'hemoglobin', 'crp', 'inr', 'ketone', 'lactate arterial', 'lactate venous', 'mean cell volume', 'arterial paco2', 'venous pco2', 'arterial ph', 'venous ph', 'platelet count', 'arterial pao2', 'potassium', 'pt', 'oxygen saturation', 'heart rate', 'respiratory rate', 'systolic bp', 'temperature', 'diastolic bp', 'oxygen flow rate', 'weight', 'fio2']}
{'static': ['M00_M99', 'D50_D89', 'F01_F99', 'K00_K95', 'S00_T88', 'V00_Y99', 'H60_H95', 'A00_B99', 'R00_R99', 'G00_G99', 'Z00_Z99', 'I00_I99', 'C00_D49', 'J00_J99', 'N00_N99', 'L00_L99', 'E00_E89', 'H0

2022-04-29 11:07:02,129 INFO cyclops.feature_handler - Saving static features to ./test_features_static.gzip
2022-04-29 11:07:02,164 INFO cyclops.feature_handler - Saving temporal features to ./test_features_temporal.gzip


## Create new FeatureHandler and load features from file.

In [4]:
# feature_handler = FeatureHandler()
# feature_handler.load(".", "test_features")